# Analisi generale progetto Big Data, Alessandro Stefani, Cristi Gutu.
### Il progetto tratta il problema della classificazione nella rispettiva categoria di news reperite dall'agenzia ANSA

#### Importazione funzioni processamento articoli

In [1]:
from AC import preproc
from AC import get_news
import inspect
import nltk
nltk.download("stopwords")

%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cristy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Ottenimento articoli per ogni categoria

In [2]:
economia = get_news("./articoli_economia/")
cultura = get_news("./articoli_cultura/")
tech = get_news("./articoli_tech/")
politica = get_news("./articoli_politica/")
sport = get_news("./articoli_sport/")
cronaca = get_news("./articoli_cronaca/")

for articolo in economia:
    articolo['categoria'] = "Economia"
for articolo in cultura:
    articolo['categoria'] = "Cultura"
for articolo in tech:
    articolo['categoria'] = "Tech"
for articolo in politica:
    articolo['categoria'] = "Politica"
for articolo in sport:
    articolo['categoria'] = "Sport"
for articolo in cronaca:
    articolo['categoria'] = "Cronaca"
dati_preprocessati =  preproc(tech + politica + cultura + economia + sport + cronaca)

100%|██████████| 2400/2400 [00:17<00:00, 134.37it/s]


In [3]:
from AC import distribuzione_frequenze

### Importazione moduli necessari per la suddivisione dell'insieme di news in train e test

In [4]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn import linear_model,metrics
from sklearn.dummy import DummyClassifier 
from sklearn.tree import DecisionTreeClassifier


### Suddivisione news in insieme di training e test

In [5]:
seed = 2
train_documents,test_documents = train_test_split(dati_preprocessati,random_state=seed, train_size = 0.5)

C:\Users\Cristy\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


### Costruzione della term document matrix e calcolo distribuzione Document Frequency

In [6]:
docs_texts = [' '.join([word for word in x['testo']] + x['tags'] + x['sottotitolo'] + x['titolo_articolo']) for x in dati_preprocessati]
train_texts = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in train_documents]
test_texts = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in test_documents]

docs_cats = [x["categoria"] for x in dati_preprocessati]
train_cats = [x["categoria"] for x in train_documents]
test_cats = [x["categoria"] for x in test_documents]


#distribuzione_frequenze(test_texts,"test",ngrammi=(1,6),min_df=11)
print(minf)

NameError: name 'minf' is not defined

###### Nella figura si vede la distribuzione del numero di volte che una parola comapre nei documenti di training e di test
##### Esempio: 6 parole compaiono in piu di 500 documenti

# RANDOM SEARCH

In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import ParameterSampler
import tqdm

train_documents,test_documents = train_test_split(dati_preprocessati,random_state=2, train_size = 0.5)
test_documents,val_documents = train_test_split(test_documents,random_state=2, train_size = 0.5)

train_texts = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in train_documents]
test_texts = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in test_documents]
val_texts = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in val_documents]

train_cats = [x["categoria"] for x in train_documents]
test_cats = [x["categoria"] for x in test_documents]
val_cats = [x["categoria"] for x in val_documents]
    
ldac = Pipeline([
    ("count_mx",CountVectorizer(encoding='utf-8',max_features=1000000, lowercase=True)),
    ("lda", LatentDirichletAllocation(max_iter=50, learning_method='online',random_state=0)), 
    ("classifier",DecisionTreeClassifier(random_state=0 ))
])

param_grid = ParameterGrid({
    'classifier__max_depth': np.arange(14, 18),
    'classifier__min_samples_leaf':[1],# 2 ** np.arange(0,3),
    'lda__n_components':[6, 12, 24],
    'lda__learning_decay':[.5, .7, .9],
    'count_mx__ngram_range':[(1,i) for i in range(1,15)],
    'count_mx__min_df': np.arange(1, 25),
    'count_mx__max_df': [60,70,0.5,0.6,0.7,0.8,0.9]
})
#print(param_grid.param_grid)
param_list = list(ParameterSampler(param_grid.param_grid[0],n_iter=10,random_state=0))



In [ ]:
print(param_list)

In [ ]:
risultati = []

for params in tqdm.tqdm(param_list):
    ldac.set_params(**params)
    ldac.fit(train_texts, train_cats)
    y_pred = ldac.predict(val_texts)
    params["accuracy_score"] = metrics.accuracy_score(val_cats, y_pred)
    risultati.append(params)


In [ ]:
risultati = pd.DataFrame(risultati).sort_values(["accuracy_score",'count_mx__min_df','count_mx__max_df','lda__n_components','lda__learning_decay', "classifier__max_depth",], ascending=[False, True, True, True, True, True])
risultati.reset_index(drop=True, inplace=True)
print("Primi 5:")
display(risultati.head(20))

print("Ultimi 5:")
risultati.tail(10)

In [ ]:
params = {'lda__n_components': 6, 'lda__learning_decay': 0.7, 'count_mx__ngram_range': (1, 12), 'count_mx__min_df': 14, 'count_mx__max_df': 70, 'classifier__min_samples_leaf': 1, 'classifier__max_depth': 17}
#{'lda__n_components': 8, 'lda__learning_decay': 0.9, 'count_mx__ngram_range': (1, 2), 'count_mx__min_df': 13, 'count_mx__max_df': 0.7, 'classifier__min_samples_leaf': 1, 'classifier__max_depth': 17}

In [ ]:
ldac.set_params(**params)
ldac.fit(train_texts, train_cats)

In [ ]:
t1,t2 = train_test_split(test_documents,random_state=42, train_size = 0.5)
t1c = [x["categoria"] for x in t1]
t2c = [x["categoria"] for x in t2]
t1 = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in t1]
t2 = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in t2]

In [ ]:
y_pred = ldac.predict(t1)
params["accuracy_score"] = metrics.accuracy_score(t1c, y_pred)
ris=[params]
ris = pd.DataFrame(ris)
display(ris)

In [ ]:
y_pred = ldac.predict(t2)
params["accuracy_score"] = metrics.accuracy_score(t2c, y_pred)
ris=[params]
ris = pd.DataFrame(ris)
display(ris)

In [ ]:
y_pred = ldac.predict(test_texts)
params["accuracy_score"] = metrics.accuracy_score(test_cats, y_pred)
ris=[params]
ris = pd.DataFrame(ris)
display(ris)

# Accuratezza al variare della dimensione del training set con e senza LDA

In [7]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import tqdm


In [12]:
ldac = Pipeline([
    ("count_mx",CountVectorizer(encoding='utf-8',max_features=1000000, lowercase=True)),
    ("classifier",DecisionTreeClassifier(random_state=0))
])
params = {
    'count_mx__ngram_range': (1, 12),
    'count_mx__min_df': 13,
    'count_mx__max_df': 0.9,
    'classifier__min_samples_leaf': 1,
    'classifier__max_depth': 15
}
ldac.set_params(**params)

res = []
for i in [0.8]:
    train_documents,test_documents = train_test_split(dati_preprocessati,random_state=2, train_size = i)

    train_texts = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in train_documents]
    test_texts = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in test_documents]

    train_cats = [x["categoria"] for x in train_documents]
    test_cats = [x["categoria"] for x in test_documents]
    
    ldac.fit(train_texts, train_cats)
    y_pred = ldac.predict(test_texts)
    res.append({"accuracy_score":metrics.accuracy_score(test_cats, y_pred),"train_size": i})
res = pd.DataFrame(res).sort_values(["train_size","accuracy_score"], ascending=[True,False])
display(res)

C:\Users\Cristy\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


,accuracy_score,train_size
0,0.85,0.8


In [42]:
ldac = Pipeline([
    ("count_mx",CountVectorizer(encoding='utf-8',max_features=1000000, lowercase=True)),
    ("lda", LatentDirichletAllocation(max_iter=50, learning_method='online',random_state=0)),
    ("classifier",DecisionTreeClassifier(random_state=0))
])
params = {
    'count_mx__ngram_range': (1, 8),
    'count_mx__min_df': 13,
    'count_mx__max_df': 0.9,
    'lda__n_components':8,
    'lda__learning_decay':0.5,
    'classifier__min_samples_leaf': 1,
    'classifier__max_depth': 15
}
ldac.set_params(**params)

res = []
for i in [0.1,0.2,0.3,0.4,0.5,0.6,0.75]:
    train_documents,test_documents = train_test_split(dati_preprocessati,random_state=2, train_size = i)

    train_texts = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in train_documents]
    test_texts = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in test_documents]

    train_cats = [x["categoria"] for x in train_documents]
    test_cats = [x["categoria"] for x in test_documents]

    ldac.fit(train_texts, train_cats)
    pred_cats = ldac.predict(test_texts)
    res.append({"accuracy_score":metrics.accuracy_score(test_cats, pred_cats),"train_size": i})
res = pd.DataFrame(res).sort_values(["train_size","accuracy_score"], ascending=[True,False])
display(res)

C:\Users\Cristy\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
C:\Users\Cristy\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
C:\Users\Cristy\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
C:\Users\Cristy\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
C:\Users\Cristy\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both ar

,accuracy_score,train_size
0,0.564352,0.10
1,0.769792,0.20
2,0.885119,0.30
3,0.913194,0.40
4,0.977500,0.50
5,0.980208,0.60
6,0.993333,0.75


In [32]:
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,classification_report
from itertools import product

In [33]:
print(classification_report(test_cats, pred_cats, target_names=unique_labels(test_cats, pred_cats)))

              precision    recall  f1-score   support

     Cronaca       1.00      1.00      1.00       127
     Cultura       1.00      1.00      1.00       109
    Economia       0.96      0.98      0.97       118
    Politica       1.00      0.96      0.98       122
       Sport       1.00      1.00      1.00       119
        Tech       0.98      1.00      0.99       125

   micro avg       0.99      0.99      0.99       720
   macro avg       0.99      0.99      0.99       720
weighted avg       0.99      0.99      0.99       720



# TEMPI

In [37]:
seed = 2
train_documents,test_documents = train_test_split(dati_preprocessati,random_state=seed, train_size = 0.75)

train_texts = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in train_documents]
test_texts = [' '.join([word for word in x['testo']] + x['sottotitolo'] + x['titolo_articolo']) for x in test_documents]

train_cats = [x["categoria"] for x in train_documents]
test_cats = [x["categoria"] for x in test_documents]

C:\Users\Cristy\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [38]:
tf = CountVectorizer(ngram_range=(1,2), min_df=13,max_df=0.7,max_features=1000000, lowercase=True).fit(train_texts)
tf_train = tf.transform(train_texts)
ldax = LatentDirichletAllocation(n_components=8,max_iter=50,learning_decay=0.9, learning_method='online',random_state=0).fit(tf_train.toarray())
ldax_train = ldax.transform(tf_train.toarray())
dt = DecisionTreeClassifier(random_state=0, min_samples_leaf=1,max_depth=17).fit(ldax_train,train_cats)

tf_test = tf.transform(test_texts)
ldax_test = ldax.transform(tf_test.toarray()) 

In [39]:
import timeit

REPEAT = 30
NUMBER = 1
tempi = timeit.repeat("dt.predict(ldax_test)",setup="from __main__ import dt,ldax_test",repeat=REPEAT, number=NUMBER)
print("{:.2f} s ± {:.0f} ms per loop (mean ± std. dev. of {} runs, {} loop each)".format(
    np.mean(tempi), 1000 * np.std(tempi), REPEAT, NUMBER))

0.00 s ± 0 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [40]:
tf = CountVectorizer(ngram_range=(1,2), min_df=13,max_df=0.7,max_features=1000000, lowercase=True).fit(train_texts)
tf_train = tf.transform(train_texts)
tf_train = tf_train.toarray()
dt = DecisionTreeClassifier(random_state=0, min_samples_leaf=1,max_depth=17).fit(tf_train,train_cats)

tf_test = tf.transform(docs_texts)
tf_test = tf_test.toarray()

In [41]:
import timeit

REPEAT = 30
NUMBER = 1
tempi = timeit.repeat("dt.predict(tf_test)",setup="from __main__ import dt,tf_test",repeat=REPEAT, number=NUMBER)

print("{:.2f} s ± {:.0f} ms per loop (mean ± std. dev. of {} runs, {} loop each)".format(
    np.mean(tempi), 1000 * np.std(tempi), REPEAT, NUMBER))

0.02 s ± 1 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)
